### Installation

This section handles the installation of necessary Python libraries. These libraries are crucial for building the RAG (Retrieval Augmented Generation) and hybrid agents:

-   **`langchain`**: The core framework for developing applications powered by language models. It provides tools and components for building complex LLM workflows.
-   **`langchain-openai`**: The integration package for using OpenAI's models (both chat and embeddings) with LangChain.
-   **`langchain-text-splitters`**: Provides utilities for splitting large documents into smaller, manageable chunks, which is essential for effective retrieval.
-   **`langchain-community`**: Contains various community-contributed LangChain components, including document loaders like `WebBaseLoader`.
-   **`bs4` (BeautifulSoup4)**: A library used for parsing HTML and XML documents, specifically for web scraping to extract content from web pages.
-   **`langchain-tavily`**: The integration for using Tavily Search, a web search API, as a tool within LangChain agents for real-time information retrieval.

In [1]:
!pip install -U -q langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 9.0 MB/s eta 0:00:00


In [2]:
# Installing the OpenAI integration
!pip install -U -q langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.9 MB/s eta 0:00:00


In [3]:
!pip install -q langchain-text-splitters langchain-community bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
!pip install -U -q "langchain[openai]"

### Chat Model

A **Chat Model** is a type of Large Language Model (LLM) specifically fine-tuned for conversational interactions. It's designed to understand and generate human-like text in a dialogue format. In this notebook, `ChatOpenAI` is used to interface with OpenAI's chat models (e.g., `gpt-4o`) to power the agent's conversational abilities and decision-making.

In [5]:
import os
from langchain_openai import ChatOpenAI
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

model = ChatOpenAI(model="gpt-4o")

### Embeddings Model

An **Embeddings Model** converts text (words, phrases, or documents) into numerical vector representations (embeddings) in a high-dimensional space. Texts with similar meanings are represented by vectors that are close to each other in this space. These embeddings are fundamental for:

-   **Semantic Search**: Finding documents or text snippets that are semantically related to a query.
-   **Retrieval**: Identifying relevant pieces of information from a large corpus.

Here, `OpenAIEmbeddings` with a model like `text-embedding-3-large` is used to create these numerical representations for the documents.

In [6]:
# OpenAI Embeeding Model
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

### Vector Store

A **Vector Store** (or Vector Database) is a database optimized for storing and querying vector embeddings. It allows for efficient similarity search, meaning you can quickly find vectors (and thus the corresponding text) that are most similar to a given query vector.

In this notebook, two types are shown:

-   **`InMemoryVectorStore`**: A simple vector store that keeps all embeddings and their associated data in memory. It's suitable for smaller datasets or for quick prototyping.
-   **`Chroma` (commented out)**: A more robust, persistent vector database that can store data on disk, making it suitable for larger-scale applications where data needs to be saved and reloaded.

In [7]:
!pip install -U -q "langchain-core"

In [8]:
# In Memory Vector Store
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [ ]:
# ChromaDB Vector Store

In [ ]:
# !pip install -qU langchain-chroma

In [ ]:
# from langchain_chroma import Chroma

# vector_store = Chroma(
#     collection_name="example_collection",
#     embedding_function=embeddings,
#     persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
# )

### Indexing

**Indexing** is the process of preparing and storing documents in a way that allows for fast and efficient retrieval. This typically involves several steps:

##### Loading Data

This step involves fetching the raw data that will be used. `WebBaseLoader` is used here to download content from a specified URL. `bs4.SoupStrainer` is employed to filter the HTML content, ensuring only relevant parts (like post titles, headers, and content) are extracted, thus focusing on valuable information and reducing noise.

In [9]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 43047


In [10]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


#### Splitting documents

Large documents are often too long to be processed effectively by LLMs or to be stored as single chunks in a vector store. **Text splitting** breaks these documents into smaller, overlapping segments (chunks). `RecursiveCharacterTextSplitter` is used because it attempts to split text in a smart way (e.g., by paragraphs, then sentences, then words) and includes `chunk_overlap` to maintain context across splits. `chunk_size` defines the maximum length of each segment.

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 63 sub-documents.


#### Storing Document

Once documents are split and embedded, they are added to the vector store. The `add_documents` method of the `vector_store` takes the split documents, generates their embeddings using the configured `embeddings` model, and then stores these embeddings (along with their metadata) for future retrieval.

In [12]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['5daf6841-a2a7-437e-b804-20233795eb89', '87d57de3-9e4b-4d0a-b997-4da4354c020d', '9ff0aad9-372e-4e42-ab92-22ff3d5ac722']


### RAG Agent

A **RAG (Retrieval Augmented Generation) Agent** is an LLM-based agent that enhances its responses by first retrieving relevant information from a knowledge base (like a vector store) before generating an answer. This approach helps ground the LLM's responses in factual data, reducing hallucinations and improving accuracy.

#### Retrieval Tool (`retrieve_context`)
The `@tool` decorator transforms a Python function into a tool that an LLM agent can use. The `retrieve_context` tool's purpose is to query the `vector_store` using semantic similarity search. It takes a user's query, finds the most relevant document chunks from the indexed data, and returns them as context for the LLM.

In [13]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

#### Agent Creation
`create_agent` constructs the RAG agent. It takes the `ChatOpenAI` model, the list of available tools (initially just `retrieve_context`), and a `system_prompt`. The `system_prompt` is crucial as it defines the agent's role, constraints, and behavior, instructing it to *exclusively* use the provided retrieval tool for answers and to admit limitations if context is unavailable.

In [14]:
from langchain.agents import create_agent

tools = [retrieve_context]
# If desired, specify custom instructions
prompt = """You are a retrieval-based assistant that answers questions EXCLUSIVELY using information obtained from the provided retrieval tool. You must follow these rules strictly:

## CORE RULES:
1. **USE ONLY TOOL-PROVIDED INFORMATION**: You may ONLY answer questions using information that is explicitly present in the context retrieved by the tool.
2. **NO EXTERNAL KNOWLEDGE**: You must NOT use any information from your training data, general knowledge, or any source outside the tool's retrieved context.
3. **VERBATIM ACCURACY**: When answering, stay faithful to the exact information in the retrieved context. Do not infer, extrapolate, or add information.
4. **ADMISSION OF LIMITATIONS**: If the tool does not retrieve relevant information for a query, respond ONLY with: "I have no context for this."

## RESPONSE GUIDELINES:
- DO: Quote or paraphrase directly from retrieved context
- DO: Say "I have no context for this" when information is not available
- DO: Ask for clarification if the query is ambiguous relative to available context
- DON'T: Add information from your general knowledge
- DON'T: Make assumptions beyond the retrieved text
- DON'T: Answer partially if only some information is available
- DON'T: Provide explanations, analogies, or examples not in the retrieved context

## VERIFICATION PROCESS:
Before responding, verify:
1. Is the information I'm about to share explicitly stated in the retrieved context?
2. Am I adding any information from my own knowledge?
3. If the answer to #1 is NO or #2 is YES, respond with: "I have no context for this."
"""

In [15]:
agent = create_agent(model, tools, system_prompt=prompt)

In [17]:
query = (
    "What is the standard method for Task Decomposition?"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the standard method for Task Decomposition?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_8uThYkVENvuYwIukwL6zOUSf)
 Call ID: call_8uThYkVENvuYwIukwL6zOUSf
  Args:
    query: standard method for Task Decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578}
Content: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes t

In [18]:
query = (
    "Who is the President of USA?"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query, }]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Who is the President of USA?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_9IcYr9aUzRjI7rUk16XkxBBW)
 Call ID: call_9IcYr9aUzRjI7rUk16XkxBBW
  Args:
    query: President of the United States 2023
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 40518}
Content: Or
@article{weng2023agent,
  title   = "LLM-powered Autonomous Agents",
  author  = "Weng, Lilian",
  journal = "lilianweng.github.io",
  year    = "2023",
  month   = "Jun",
  url     = "https://lilianweng.github.io/posts/2023-06-23-agent/"
}
References#
[1] Wei et al. “Chain of thought prompting elicits reasoning in large language models.” NeurIPS 2022
[2] Yao et al. “Tree of Thoughts: Dliberate Problem Solving with Large Language Mo

### Web Search Tool

A **Web Search Tool** extends an agent's capabilities by allowing it to access real-time information from the internet. This is particularly useful for queries that require up-to-date facts not present in the agent's pre-indexed knowledge base.

#### `TavilySearch`
`TavilySearch` is an integration that provides web search functionality. It can be configured to fetch a certain number of results, include images, and specify search depth or time ranges. When the agent uses `tavily_tool`, it sends a query to Tavily, which returns search results that the agent can then use to formulate its answer.

In [19]:
!pip install -q langchain-tavily

In [20]:
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [21]:
from langchain_tavily import TavilySearch

tavily_tool = TavilySearch(
    max_results=20,
    topic="general",
    include_answer=False,
    # include_raw_content=True,
    include_images=True,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

/usr/local/lib/python3.12/dist-packages/langchain_tavily/tavily_research.py:97: UserWarning: Field name "output_schema" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]
/usr/local/lib/python3.12/dist-packages/langchain_tavily/tavily_research.py:97: UserWarning: Field name "stream" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]


In [22]:
webSearch_prompt = """You are a search assistant that answers questions using the web serach with the help of the  tavily_tool."""

In [23]:
webSearch_agent = create_agent(model, [tavily_tool], system_prompt=webSearch_prompt)

In [24]:
query = "What was the weather in london on 17 Nov 2025?"

for step in webSearch_agent.stream(
    {"messages": query},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What was the weather in london on 17 Nov 2025?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_K21LiAVmtCgBifMdiYJnF2h5)
 Call ID: call_K21LiAVmtCgBifMdiYJnF2h5
  Args:
    query: London weather 17 Nov 2025
    time_range: year
================================= Tool Message =================================
Name: tavily_search

{"query": "London weather 17 Nov 2025", "follow_up_questions": null, "answer": null, "images": ["https://media-eu.invisioncic.com/k333824/monthly_2025_11/FB_IMG_1763310012490.jpg.d88450ca1b9a9a8d4990fac9c0eb245a.jpg", "https://www.londoninfoguide.com/images/visiting-london-november.jpg", "https://media.timeout.com/images/106221325/750/422/image.jpg", "https://www.theweatheroutlook.com/images/maps/day1_24.png?w=288&h=300&colors=128&mode=stretch", "https://media-eu.invisioncic.com/k333824/monthly_2025_11/IMG_20251115_

### Let's do changes in our first Agent

A **Hybrid Agent** combines multiple information sources and tools to provide more comprehensive and robust answers. In this notebook, the hybrid agent integrates both the internal `retrieve_context` (RAG) tool and the external `tavily_tool` (web search).

#### Updated Prompt
The `system_prompt` for the hybrid agent is carefully crafted to define a clear hierarchy:

1.  **Prioritize Retrieval**: The agent must first attempt to answer using its internal knowledge base (via `retrieve_context`).
2.  **Fallback to Web Search**: *Only if* the retrieval tool yields no relevant information, the agent is permitted to use the `tavily_tool` for web search.

This prompt ensures that the agent leverages its specialized, indexed knowledge when available, but can also access the broader, real-time internet when necessary, without hallucinating or stating external knowledge. It also enforces strict rules about using *only* tool-provided information and admitting limitations when no context is found.

In [25]:
prompt = """You are a retrieval-based assistant that answers questions EXCLUSIVELY using information obtained from the provided tools. You have access to a retrieval tool (for searching internal/local documents) and a tavily_tool (for web search). You must follow these rules strictly:

## CORE RULES:
1. **TOOL SELECTION AUTONOMY:** Assess each query and choose the most appropriate tool:
   - Use the **retrieval tool** for questions about internal documents, stored content, or previously provided information
   - Use the **tavily_tool** for current events, real-time information, general web facts, or when the query clearly requires internet search
   - You may use your judgment to determine which tool is most likely to contain relevant information

2. **FALLBACK MECHANISM:** If your initial tool choice provides no relevant or satisfactory information, use the alternative tool as a fallback before concluding no information is available.

3. **SINGLE AGENT RESPONSE:** Only one agent (you) should respond, regardless of which tool provides the information. Do not refer to multiple agents or sources in your response.

4. **USE ONLY TOOL-PROVIDED INFORMATION:** You may ONLY use information explicitly found in the tool-provided content, whether from retrieval or tavily_tool.

5. **NO EXTERNAL KNOWLEDGE:** You must NOT use any information from your training data, general knowledge, or any other source outside the tool-provided context.

6. **VERBATIM ACCURACY:** Stay faithful to the exact information in the retrieved or web-searched context. Do not infer, extrapolate, or add information.

7. **ADMISSION OF LIMITATIONS:** If neither tool provides relevant information after attempting both (when appropriate), respond ONLY with: "I have no context for this."

## TOOL USAGE STRATEGY:
- Analyze the query nature first
- Select the tool most likely to contain the answer
- If insufficient results, try the alternative tool
- Only conclude with "no context" after exhausting appropriate options

## CLARIFICATION PROTOCOL:
- If the user's query is **ambiguous**, **underspecified**, or **contextually incomplete**, ask up to **one concise clarification question** *before* deciding on tool usage.
- Typical triggers include unspecified entities ("yesterday's match winner"), missing scope ("latest report"), or unclear subjects ("their performance").
- After receiving clarification, proceed with tool selection based on the clarified query.

## RESPONSE GUIDELINES:
- **DO:** Quote or paraphrase directly from tool-provided context.
- **DO:** State "I have no context for this" when no relevant information is available from either source.
- **DO:** Use your judgment to select the most appropriate tool first.
- **DO:** Try alternative tools if initial results are unsatisfactory.
- **DON'T:** Add information from general knowledge.
- **DON'T:** Make assumptions beyond the retrieved or web-searched text.
- **DON'T:** Provide explanations, analogies, or examples not in the context.
- **DON'T:** Answer partially if only some relevant data is available.
- **DON'T:** Mention tool names explicitly to the user.

## VERIFICATION PROCESS:
Before responding, verify:
1. Did I choose the most appropriate tool for this query type?
2. Is the information I'm about to share explicitly stated in the tool-provided context?
3. Should I try the alternative tool for better results?
4. Am I adding any information from my own knowledge?
5. If no relevant information from any tool, respond with: "I have no context for this."
"""

In [26]:
tools = [retrieve_context, tavily_tool]

In [27]:
hybrid_agent = create_agent(model, tools, system_prompt=prompt)

In [28]:
query = (
    "What is the standard method for Task Decomposition?"
)

for event in hybrid_agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the standard method for Task Decomposition?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_4KFZquGPg4KXefFfdUaeDTvP)
 Call ID: call_4KFZquGPg4KXefFfdUaeDTvP
  Args:
    query: standard methods for Task Decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578}
Content: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes 

In [29]:
query = (
    "Who is the President of USA?"
)

for event in hybrid_agent.stream(
    {"messages": [{"role": "user", "content": query, }]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Who is the President of USA?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_Qcjfe6vIieiqVzgzpkRdaD6q)
 Call ID: call_Qcjfe6vIieiqVzgzpkRdaD6q
  Args:
    query: current President of the USA 2023
    time_range: year
    topic: news
================================= Tool Message =================================
Name: tavily_search

{"query": "current President of the USA 2023", "follow_up_questions": null, "answer": null, "images": ["https://assets.publishing.service.gov.uk/government/uploads/system/uploads/image_data/file/186536/s960_Biden.png", "https://people.com/thmb/HCe0rzp-4h-SVW9Kt_gfTbdQasM=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc():focal(704x399:706x401)/Joe-Biden_01-8e74a4ee45144803be041018a2a32f87.jpg", "https://digitalchew.com/wp-content/uploads/2023/10/President-Joe-Biden.jpg", "https://m.media-amazon.com/imag

### Interactive Chat Function

The `chat_with_agent` function provides a simple command-line interface to interact with the created agent. It continuously prompts the user for input, sends the conversation history to the agent, streams the agent's response, and updates the conversation history. This allows for a dynamic and interactive demonstration of the agent's capabilities.

In [30]:
def chat_with_agent(hybrid_agent):
    print("Start chatting with the agent (type 'bye' to exit)\n")

    messages = []

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == "bye":
            print("Agent: Goodbye!")
            break

        # Add the user message to the conversation history
        messages.append({"role": "user", "content": user_input})

        print("Agent: ", end="", flush=True)
        response_text = ""

        # Stream the model’s response
        for event in hybrid_agent.stream({"messages": messages}, stream_mode="values"):
            if "messages" in event and event["messages"]:
                msg = event["messages"][-1]
                if hasattr(msg, "pretty_print"):
                    msg.pretty_print()
                elif isinstance(msg, dict):
                    content = msg.get("content", "")
                    print(content, end="", flush=True)
                    response_text += content

        print()  # newline after full response

        # Add the assistant's reply to the conversation history
        messages.append({"role": "assistant", "content": response_text})

In [31]:
chat_with_agent(hybrid_agent)

Start chatting with the agent (type 'bye' to exit)

You: What is task decomposition?
Agent: ================================ Human Message =================================

What is task decomposition?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_iSNT0kmedDYk4g1hBDm1ROv7)
 Call ID: call_iSNT0kmedDYk4g1hBDm1ROv7
  Args:
    query: task decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578}
Content: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do